In [18]:
import itertools
import pdb
import os
from helper import *

In [19]:
def findsubsets(s, n): 
    #return list(itertools.combinations(s, n)) 
    return list(map(set, itertools.combinations(s, n))) 
s = {1, 2, 3} 
n = 2
  
print(findsubsets(s, n))

[{1, 2}, {1, 3}, {2, 3}]


In [20]:
def mes(apelido_fundo,anomes,tipo):
    if tipo == 'acoes':
        aux = pd.read_csv('excel_fundos\\'+apelido_fundo+'\\'+str(anomes)+'.xls',index_col = 0,sep='\t',decimal= '.')
        lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                         'Ações e outros TVM cedidos em empréstimo',
                         'Ações',
                         'Valores a pagar']
        
        aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        #aux.pf_quant = aux.pf_quant*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        #aux.pf_valor_mercado = aux.pf_valor_mercado*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        return aux[aux.ativo1.isin(lista_acoes)]

In [21]:
def precos(anomes,ativo):
    precos = pd.read_csv(str(anomes)[:-2]+'.txt',index_col=False,sep=';',header=0,chunksize=10000).read()
    precos = precos[(precos.CODNEG == ativo)&(precos.TPMERC == 10.0)&(precos.DATA > 100*int(anomes))&(precos.DATA < 100+100*int(anomes))]
    return precos

In [22]:
def teste(carteira_0,carteira_1,ativo,anomes):
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    carteira_1 = completa(ativo,lista[0],carteira_1)
    return carteira_1

In [23]:
def teste_fraco(carteira_0,carteira_1,ativo,anomes):
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        carteira_1 = completa(ativo,lista[0],carteira_1)
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    return carteira_1

In [24]:
def filtro_alfabetico(carteira,alfabetico,aserestudado_com_varios_ativo3=[]):
    if type(aserestudado_com_varios_ativo3) != list:
        aserestudado_com_varios_ativo3 = [aserestudado_com_varios_ativo3]
    teste = carteira.copy()
    for aserestudado,ativo3,grupo in teste[~teste.ativo3.isnull()][~teste.aserestudado.isin(aserestudado_com_varios_ativo3)][['aserestudado','ativo3','grupo']].values:
    #for aserestudado,ativo3,grupo in teste[['aserestudado','ativo3','grupo']].values:
        if alfabetico:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
        else:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
    return teste
    

In [25]:
def filtro_unico_index(carteira,aserestudado_sem_nenhum_ativo3=[]):
    carteira = carteira.copy()
    if type(aserestudado_sem_nenhum_ativo3) != list:
        aserestudado_sem_nenhum_ativo3 = [aserestudado_sem_nenhum_ativo3]
    CONFUSAO = carteira[~carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3)].groupby(by='aserestudado').count()[['ativo3']]
    if (CONFUSAO.ativo3==1).all():
        print('trabalho concluido'.upper())
        return carteira[(carteira.ativo3.notnull())|(carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3))].sort_values(['aserestudado','candidatos'])
    else:
        
        print(CONFUSAO[CONFUSAO.ativo3>1])
        print(CONFUSAO[CONFUSAO.ativo3==0])
        return carteira.sort_values(['aserestudado','candidatos'])

In [26]:
def separa_blocos(carteira_pandas):
    
    
    
    teste = carteira_pandas.sort_values(by='aserestudado').copy()


    assert len(teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].index )==1
    assert len(teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].index )==1



    alfabetico = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0] < teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0] 

    if not alfabetico:
        teste =teste.sort_values(by=['candidatos'],ascending=False).sort_values(by=['aserestudado'])
    
    
    assert len(teste[teste.ativo1.str.upper() == 'Obrigações por ações e outros TVM recebidos em empréstimo'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Valores a pagar'.upper()].index )<=1

    disponibilidade = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0]
    valor_receber =teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0]


    teste['grupo'] = 0
    
    if not alfabetico:
        print('ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)')
        print('Tento by passar usando a localização de algum Investimento no Exterior')
        #return

        investimento_exterior_aserestudado =teste[teste.ativo1.str.upper() == 'Investimento no Exterior'.upper()].aserestudado.values[0]



        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4

    else:
        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4





    for t in set(teste.grupo) :
        if t != 0 and teste[teste.grupo==t].ativo3.apply(pd.isna).all() :
            print('grupo',t,'todos os ativos1 são nulos')

    return teste,alfabetico

In [30]:
anomes = 202004
apelido_fundo = 'velt'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.3 s


In [31]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)
carteira_pandas[carteira_pandas.grupo == 1]
aux = carteira_pandas.copy()

True

In [33]:
anomes = 202005
apelido_fundo = 'velt'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.12 s


In [34]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior
grupo 2 todos os ativos1 são nulos


In [36]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Mercado Futuro - Posições vendidas',
 'Outras aplicações',
 'Valores a pagar',
 'Valores a receber'}

In [204]:
def explodir(carteira_pandas,alfabetico):
    teste = carteira_pandas[carteira_pandas.grupo!=0]
    assert len(teste[teste.candidatos.isnull()]) ==0
    teste = pd.DataFrame(columns=['index','aserestudado','ativo1','ativo2','ativo3','ativo4','classificacao','emp_ligada','vendas_quant','vendas_montante','aquis_quant','aquis_montante','pf_quant','pf_valor_custo','pf_valor_mercado','porcent_partrimonio_liquido','fechado','candidatos','cand_pf_quant', 'grupo'],data=[(index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,c,q,grupo) if pd.isnull(ativo3) else (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,ativo3,pf_quant,grupo) for (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo) in teste.reset_index().values  for (c,q) in zip(candidatos,cand_pf_quant) ]).set_index('index')
    teste = teste.drop_duplicates().sort_values(by='aserestudado').reset_index().rename(columns={'index':'position'})
    

    teste = filtro_alfabetico(teste,alfabetico)
    teste.pf_quant = teste.cand_pf_quant
    
    
    teste.pf_quant = teste.pf_quant.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    teste.pf_valor_mercado = teste.pf_valor_mercado.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    
    
    
    return teste

In [38]:
explodido = explodir(carteira_pandas,alfabetico)

In [39]:
def completa(ativo,linhas,teste):
    for i in linhas:
        teste.loc[int(i),'ativo3'] = ativo
    return teste

In [40]:
lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                 'Ações e outros TVM cedidos em empréstimo',
                 'Ações',
                 'Certificado ou recibo de depósito de valores mobiliários', 
                 'Valores a pagar']

aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux =  aux[aux.ativo1.isin(lista_acoes)]


In [45]:
aux.ativo3.unique()

array(['HGTX3', 'MGLU3', 'ANIM3', 'AZUL4', 'B3SA3', 'BIDI11', 'BRML3',
       'CSAN3', 'CVCB3', 'ENEV3', 'ENGI11', 'EQTL3', 'GNDI3', 'HAPV3',
       'IGTA3', 'ITSA4', 'ITUB4', 'MILS3', 'MRVE3', 'NTCO3', 'PCAR3',
       'PETR4', 'RAIL3', 'RENT3', 'SEER3', 'SHOW3', 'SULA11', 'TEND3',
       'VALE3', 'Valores a Pagar', 'BPAC11'], dtype=object)

In [43]:
aux.loc[22,'ativo3'] = 'BPAC11'

In [47]:
MES0 = aux.copy()
MES1 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES1 = teste_fraco(MES0,MES1,ativo,'202005')

-------------------ANIM3-------------------
AZUL4 118106500.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
B3SA3 -20770000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BIDI11 128118800.0 
                                                  ativo1  aserestudado  \
1375  Certificado ou recibo de depósito de valores m...      58881820   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
1375              0.0             0.0        4719079600      1404488.0   

               MAX           MIN  
1375  1.404488e+08  1.404488e+08   
 []
BPAC11 45510000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BRML3-------------------
CSAN3 24387000.0 
 Empty DataFrame
Columns:

In [53]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [54]:
#papeis que ele Não achou
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
58881799,0
58881816,0
58881820,0


In [50]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES1 = filtro_alfabetico(MES1,alfabetico)

In [52]:

assert len(CONFUSAO[CONFUSAO.ativo3>1]) != 0
MES1 = filtro_alfabetico(MES1,alfabetico,aserestudado_com_varios_ativo3=[58881799,58881816,58881820])

In [57]:
#Não achou
nao_achou = MES1[MES1.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]

In [58]:
MES0[MES0.ativo1.str.contains('Certificado')]

,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
31,58385988,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,BIDI11,Dt. Ini. Vigen.:18/07/2019,Para negociacao,Nao,NaN,NaN,NaN,NaN,1281188.0,NaN,3525829376,1.562,False,"[A1GI34, ABEV3, ADHM3, ALUP4, ATOM3, AZEV3, AZ...","[164451, 3109197, 14217054, 5535054, 26312160,...",1
30,58385993,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,ENGI11,Dt. Ini. Vigen.:03/11/2009,Para negociacao,Nao,NaN,NaN,NaN,NaN,980227.0,NaN,4283591990,1.898,False,"[ABEV3, AMAR3, APTV34, ARZZ3, ATMP3, ATOM3, AZ...","[3777418, 6519927, 265748, 1007904, 7187235, 3...",1
29,58386011,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,SULA11,Dt. Ini. Vigen.:05/10/2007,Para negociacao,Nao,NaN,NaN,NaN,NaN,1239700.0,NaN,5547657500,2.458,False,"[A1LK34, ADHM3, AHEB3, AZEV4, BALM3, BEES3, BM...","[344575, 22369587, 2412025, 20173300, 4824050,...",1


In [ ]:
[58881799,58881816,58881820]

In [62]:
MES1[MES1.candidatos=='ENGI11']

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1097,34,58881816,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1074527.0,NaN,5104003200,1.0,True,ENGI11,1074527.0,1


In [63]:
MES1 = completa('SULA11',{89},MES1)
MES1 = completa('BIDI11',{1375},MES1)
MES1 = completa('ENGI11',{1097},MES1)
MES1 = filtro_unico_index(MES1)
MES1

TRABALHO CONCLUIDO


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
18,28,58881798,Ações,NaN,TEND3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,1.572081e+08,854180.0,NaN,2143137600,0.000,True,TEND3,854180.0,1
89,33,58881799,Certificado ou recibo de depósito de valores m...,NaN,SULA11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1359000.0,NaN,6095115000,2.000,True,SULA11,1359000.0,1
163,31,58881800,Ações,NaN,SHOW3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,0.000000e+00,1156023.0,NaN,193055800,0.000,True,SHOW3,1156023.0,1
238,29,58881801,Ações,NaN,SEER3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,1.874026e+08,1407621.0,NaN,2049496100,0.000,True,SEER3,1407621.0,1
277,26,58881802,Ações,NaN,RENT3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,1.841741e+08,660409.0,NaN,2541253800,0.000,True,RENT3,660409.0,1
346,20,58881803,Ações,NaN,RAIL3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,7.780140e+08,4398563.0,NaN,9909962400,3.000,True,RAIL3,4398563.0,1
432,3,58881804,Ações,Cod. Ativo:,PETR4,Dt. Ini. Vigen.:24/03/1997,Para negociacao,Nao,2086100.0,3.945369e+09,237800.0,4.160889e+08,1303606.0,NaN,2651534604,0.985,False,PETR4,1303606.0,1
440,23,58881805,Ações,NaN,PCAR3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,4.069561e+08,750835.0,NaN,4728758800,1.000,True,PCAR3,750835.0,1
501,14,58881806,Ações,NaN,NTCO3,NaN,Para negociacao,Nao,NaN,0.000000e+00,NaN,1.630665e+09,5007044.0,NaN,18616189500,6.000,True,NTCO3,5007044.0,1
587,19,58881807,Ações,NaN,MRVE3,NaN,Para negociacao,Nao,NaN,2.793569e+08,NaN,8.961476e+08,7115304.0,NaN,10793916100,4.000,True,MRVE3,7115304.0,1


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1106,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2394690.0,NaN,10464795300,3.0,True,ENGI11,2394690.0,1
1105,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,34423669.0,NaN,10464795300,3.0,True,EQPA3,34423669.0,1
1104,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,17470443.0,NaN,10464795300,3.0,True,EQPA7,17470443.0,1


In [30]:
MES1 = completa('ENGI11',{1106},MES1)
MES1 = filtro_unico_index(MES1)
MES1

Empty DataFrame
Columns: [ativo3]
Index: []
              ativo3
aserestudado        
58334057           0
58334058           0


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
0,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,ARZZ3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,113579.0,NaN,482710700,0.0,True,ARZZ3,113579.0,2
28,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,3602319.0,NaN,482710700,0.0,True,ATOM3,3602319.0,2
29,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,207172.0,NaN,482710700,0.0,True,BGIP4,207172.0,2
30,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2806458.0,NaN,482710700,0.0,True,BOBR4,2806458.0,2
31,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,161658.0,NaN,482710700,0.0,True,BRAP4,161658.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,23884336.0,NaN,12658698000,3.0,True,USIM3,23884336.0,3
2461,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,83280908.0,NaN,12658698000,3.0,True,VIVR3,83280908.0,3
2460,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,1346670.0,NaN,12658698000,3.0,True,VVPR11,1346670.0,3
2486,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,4366574.0,NaN,12658698000,3.0,True,WSON33,4366574.0,3


In [64]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [65]:
list(CONFUSAO[CONFUSAO.ativo3<1].index)

[]

In [66]:
MES1 = filtro_alfabetico(MES1,alfabetico)
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [67]:
MES1 = filtro_unico_index(MES1,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES1 = MES1.set_index('index')

TRABALHO CONCLUIDO


In [68]:
#MES1[MES1.aserestudado==58334057]

In [69]:
#MES1[MES1.aserestudado==58334058]

''

In [72]:
MES1.sort_values(['aserestudado','candidatos']).to_csv('excel_fundos\\'+str(anomes)+'_2.xls',sep='\t',decimal= '.')

In [73]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES1.copy()

In [74]:
anomes = 202006

%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.09 s


In [75]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior
grupo 2 todos os ativos1 são nulos


In [76]:
explodido = explodir(carteira_pandas,alfabetico)

In [77]:
MES1 = aux.copy()
MES2 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES2 = teste_fraco(MES1,MES2,ativo,'202006')

-------------------ANIM3-------------------
B3SA3 -760000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BIDI11 140448800.0 
                                                  ativo1  aserestudado  \
834                                               Ações      59631545   
1293  Certificado ou recibo de depósito de valores m...      59631552   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
834               0.0      38112000.0       13812530700      3352556.0   
1293              0.0             0.0        6666275300      1618028.0   

               MAX           MIN  
834   3.344032e+08  3.339957e+08  
1293  1.618028e+08  1.618028e+08   
 []
-------------------BRML3-------------------
-------------------CVCB3-------------------
ENEV3 443502200.0 
                                         ativo1  aserestudado  vendas_montante  \
964                                 

In [93]:
MES2 = completa('ENEV3',{1619,964},MES2)
MES2 = completa('SULA11',{17},MES2)
MES2 = completa('BIDI11',{1293},MES2)
MES2 = completa('ENGI11',{925},MES2)

In [131]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [146]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
59631550,0


In [147]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES2 = filtro_alfabetico(MES2,alfabetico)

In [148]:
#Não achou
nao_achou = MES2[MES2.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]

In [92]:
MES1[MES1.ativo1.str.contains('Certificado')]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
89,33,58881799,Certificado ou recibo de depósito de valores m...,NaN,SULA11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1359000.0,NaN,6095115000,2.0,True,SULA11,1359000.0,1
1097,34,58881816,Certificado ou recibo de depósito de valores m...,NaN,ENGI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1074527.0,NaN,5104003200,1.0,True,ENGI11,1074527.0,1
1375,35,58881820,Certificado ou recibo de depósito de valores m...,NaN,BIDI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1404488.0,NaN,4719079600,1.0,True,BIDI11,1404488.0,1


In [145]:
MES2 = teste_fraco(MES1,MES2,'BIDI3',str(anomes))

-------------------BIDI3-------------------


In [151]:
MES2 = teste_fraco(MES1,MES2,'BIDI4',str(anomes))

-------------------BIDI4-------------------


In [139]:
MES3[MES3.candidatos=='BIDI3']

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1237,30,60015343,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,4448000.0,136268.0,NaN,265313700,0.0,True,BIDI3,136268.0,1


In [152]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [153]:
MES2 = filtro_unico_index(MES2,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [52]:
MES2[MES2.aserestudado==58845455]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
2659,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,34782.0,NaN,86502800,0.0,True,ALUP11,34782.0,3
2660,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,7683.0,NaN,86502800,0.0,True,ALZR11,7683.0,3
2661,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,39445.0,NaN,86502800,0.0,True,ANIM3,39445.0,3
2662,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,20547.0,NaN,86502800,0.0,True,ARZZ3,20547.0,3
2663,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,537285.0,NaN,86502800,0.0,True,ATOM3,537285.0,3
2664,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,221235.0,NaN,86502800,0.0,True,AZEV3,221235.0,3
2665,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,247151.0,NaN,86502800,0.0,True,AZEV4,247151.0,3


In [154]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES2.copy()

In [155]:
anomes = 202007


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.09 s


In [156]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior
grupo 2 todos os ativos1 são nulos


In [157]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Mercado Futuro - Posições vendidas',
 'Opções - Posições titulares',
 'Outros valores mobiliários registrados na CVM objeto de oferta pública',
 'Valores a pagar',
 'Valores a receber'}

In [158]:
explodido = explodir(carteira_pandas,alfabetico)

202007

In [160]:
MES2 = aux.copy()
MES3 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES3 = teste_fraco(MES2,MES3,ativo,'202007')

-------------------ANIM3-------------------
B3SA3 21140000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BIDI11 161802800.0 
                                                  ativo1  aserestudado  \
1290  Certificado ou recibo de depósito de valores m...      60015344   
1491           Ações e outros TVM cedidos em empréstimo      60015348   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
1290              0.0             0.0       10508982200      1734728.0   
1491              0.0             0.0        1581434800       261049.0   

               MAX           MIN  
1290  1.734728e+08  1.734728e+08  
1491  2.610490e+07  2.610490e+07   
 []
-------------------BIDI3-------------------
-------------------BIDI4-------------------
-------------------BRML3-------------------
-------------------CVCB3-------------------
-------------------ENEV3-------------------
ENGI

In [206]:
MES3 = completa('BIDI11',{1290},MES3)
MES3 = completa('ENGI11',{945},MES3)
MES3 = completa('SULA11',{70},MES3)
MES3 = completa('SHOW3',{164},MES3)


In [207]:
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [208]:
#papeis que ele Não achou
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
60015319,0
60015321,0


In [218]:
nao_achou = MES3[MES3.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]


In [212]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES3 = filtro_alfabetico(MES3,alfabetico)

In [213]:
MES3 = filtro_unico_index(MES3,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [188]:
MES3[MES3.ativo3=='OIBR3']
MES3.loc[372, 'ativo3']=MES3.loc[372, 'ativo4']

''

In [231]:
MES3 = teste_fraco(MES2,MES3,'UGPA3','202007')

-------------------UGPA3-------------------


In [216]:
for k in nao_achou.aserestudado.unique():
    for cand in MES3[MES3.aserestudado==k].candidatos.values:
        teste_fraco(MES2,MES3,cand,'202007')

-------------------TASA13-------------------
-------------------TASA15-------------------
-------------------TASA17-------------------
-------------------TCNO3-------------------
-------------------TCNO4-------------------
-------------------TECN3-------------------
-------------------TIET3-------------------
-------------------TPIS3-------------------
-------------------UGPA3-------------------
-------------------VIVT4-------------------
-------------------VULC3-------------------
SLED3 0.0 
     ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
112  Ações      60015321              0.0    2.347345e+09        2615274700   

     cand_pf_quant           MAX           MIN  
112     18680534.0  6.943808e+08  1.546625e+08   
 []


IndexError: single positional indexer is out-of-bounds

'SULA11'

In [189]:
MES3 = teste_fraco(MES2,MES3,'RADL3','202007')

-------------------RADL3-------------------


In [65]:
MES0.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202003)+'.xls',sep='\t',decimal= '.')

In [66]:
MES1.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202004)+'.xls',sep='\t',decimal= '.')

In [76]:
MES2.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202005)+'.xls',sep='\t',decimal= '.')

In [72]:
MES3.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202006)+'.xls',sep='\t',decimal= '.')